<a href="https://colab.research.google.com/github/akhil-maker/Deep-Learning/blob/master/Lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [44]:
transform = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [45]:
train_dataset.data[0].shape

torch.Size([28, 28])

In [46]:
class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    # Calculate the input size for the first fully connected layer
    # based on the output size after the conv and pool layers.
    # Input size: 1x28x28
    # After conv1 (5x5 kernel, stride 1, padding 2): 1x32x32 -> 6x28x28 (output size calculation: (input_size - kernel_size + 2*padding)/stride + 1)
    # After pool1 (2x2 kernel, stride 2): 6x14x14
    # After conv2 (5x5 kernel, stride 1): 6x14x14 -> 16x10x10
    # After pool2 (2x2 kernel, stride 2): 16x5x5
    self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=84)
    self.fc3 = nn.Linear(in_features=84, out_features=10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool1(F.relu(x))
    x = self.conv2(x)
    x = self.pool2(F.relu(x))
    x = x.view(-1, 16*5*5) # Reshape the tensor to match the input of the first fully connected layer
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return x

In [47]:
model = LeNet5()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [49]:
for epoch in range(10):
  model.train()
  for xb, yb in train_dataloader:
    pred = model(xb)
    loss = loss_fn(pred, yb)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  model.eval()
  correct = 0
  total_loss = 0
  with torch.no_grad():
    for xb, yb in test_dataloader:
      pred = model(xb)
      loss = loss_fn(pred, yb)
      total_loss += loss.item()
      pred = torch.argmax(pred, dim=1)
      correct += (pred == yb).sum().item()
  print(f"Epoch {epoch} val_loss {total_loss} accuracy: {correct:.4f}")

Epoch 0 val_loss 6.615876127951196 accuracy: 9867.0000
Epoch 1 val_loss 6.268524947699916 accuracy: 9872.0000
Epoch 2 val_loss 7.072163025397458 accuracy: 9851.0000
Epoch 3 val_loss 6.5722986315740854 accuracy: 9855.0000
Epoch 4 val_loss 5.921918610101784 accuracy: 9874.0000
Epoch 5 val_loss 6.102289465198737 accuracy: 9888.0000
Epoch 6 val_loss 6.9384347162995255 accuracy: 9872.0000
Epoch 7 val_loss 6.023855745301262 accuracy: 9891.0000
Epoch 8 val_loss 6.915884139099489 accuracy: 9877.0000
Epoch 9 val_loss 9.113462038907528 accuracy: 9845.0000
